In [33]:
import pandas as pd
import numpy as np
from scipy.special import expit

In [25]:
defs = pd.read_csv("db/sample/clean/def_col_vectors.csv", names=["word", "word_vector","definition", "def_vectors"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
word_list = pd.read_csv("db/sample/clean/word_col_vectors.csv", names=["word", "word_vector"], sep=":", index_col=None, keep_default_na=False, na_values=[""])


X = []
y = []

defs = defs.values.tolist()
word_list = word_list.values.tolist()
defs_main = defs[:6575]

defs_main = [[str(d[0]).strip(), d[1], str(d[2]).strip(), str(d[3]).strip()] for d in defs_main]
size = len(word_list)

epoch=9000
lr=0.1

for n in defs_main:
    main_word = str(n[0])
    definition = str(n[2]).split()
    vector = str(n[3]).split()
    out = np.zeros((size,), dtype=int)
    inp = np.zeros((size,), dtype=int)
    for word in word_list:
        if word[0] is main_word:
            out[word[1]] = 1
        if word[0] in definition:
            inp[word[1]] = 1
    X.append(inp)
    y.append(out)

    
X = np.asarray(X)
y = np.asarray(y)


In [34]:
def sigmoid(x):
    return 1/(1+expit(-x))

def der_sigmoid(x):
    return 1*(1-x)

input_layer_neurons = X.shape[1] # 3
hiddenlayer_neurons = 3
output_neurons = size

wh = np.random.uniform(size = (input_layer_neurons, hiddenlayer_neurons))
bh = np.random.uniform(size = (1, hiddenlayer_neurons))
wout = np.random.uniform(size = (hiddenlayer_neurons, output_neurons))
bout = np.random.uniform(size = (1, output_neurons))

for i in range(epoch):
    # Forward propagation
    hidden_layer_input1 = np.dot(X,wh)
    hidden_layer_input = hidden_layer_input1 + bh
    hidden_layer_activations = sigmoid(hidden_layer_input)
    output_layer_input1 = np.dot(hidden_layer_activations, wout)
    output_layer_input = output_layer_input1 + bout
    output = sigmoid(output_layer_input)

    # Backpropagation
    E = y-output
    slope_output_layer = der_sigmoid(output)
    slope_hidden_layer = der_sigmoid(hidden_layer_activations)
    d_output = E * slope_output_layer
    Error_at_hidden_layer = d_output.dot(wout.T)
    d_hiddenlayer = Error_at_hidden_layer * slope_hidden_layer
    wout += hidden_layer_activations.T.dot(d_output) * lr
    bout += np.sum(d_output, axis=0, keepdims = True) * lr
    wh += X.T.dot(d_hiddenlayer) * lr
    bh += np.sum(d_hiddenlayer, axis=0, keepdims = True) * lr

print(output)

KeyboardInterrupt: 